In [1]:
# Importing libraries
import os
import subprocess 
import json
from dotenv import load_dotenv

In [2]:
# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("mnemonic")

In [3]:
# Import constants.py and necessary functions from bit and web3
from constants import *
import inspect
from bit import Key, PrivateKey, PrivateKeyTestnet
from bit import *
from bit.network import NetworkAPI
from web3 import Web3, middleware, Account
from bit import PrivateKeyTestnet
# from eth_account import Account
from getpass import getpass
from pathlib import Path
from web3.middleware import geth_poa_middleware

In [4]:
# Web3 connection and loading mnemonic
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1.8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [5]:
# Create a function called `derive_wallets`
def derive_wallets(mnemonic, coin, numderive):
    """Use the subprocess library to call the php file script from Python"""
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --numderive="{numderive}" --coin="{coin}" --format=json' 
    
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
   
    p_status = p.wait()
    keys = json.loads(output)
    return  keys
 

In [6]:
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {"eth", "btc-test"}
numderive = 3

In [7]:
# Setting the dictionarry
keys = {}
for coin in coins:
    keys[coin]= derive_wallets(os.getenv('mnemonic'), coin, numderive=3)

In [8]:
# Creating a private keys object
eth_PrivateKey = keys["eth"][0]['privkey']
btc_PrivateKey = keys['btc-test'][0]['privkey']


print(json.dumps(eth_PrivateKey, indent=4, sort_keys=True))
print(json.dumps(btc_PrivateKey, indent=4, sort_keys=True))
print(json.dumps(keys, indent=4, sort_keys=True))

"0x31a708a269e7c8f9cbb934afe97001f542e0e791458a8848ccd6c4040efef689"
"cULobjtujJ4xn28zYUpu5bji5yW6XDpHGeT7ufafS8z6UPEU19Km"
{
    "btc-test": [
        {
            "address": "mwaPpwmv4RHGKpHBdaJZf4RcTxEZBKDKqr",
            "index": 0,
            "path": "m/44'/1'/0'/0/0",
            "privkey": "cULobjtujJ4xn28zYUpu5bji5yW6XDpHGeT7ufafS8z6UPEU19Km",
            "pubkey": "030efafe05f8ee8de6071c787cc1782e68fcfc0fdbdb9d3ce5fa7fa2d54607c0fc",
            "pubkeyhash": "b02892efdb9fc0f37b4bab1efde62d47e137fef6",
            "xprv": "tprv8jBu9RBfFdhQZmi2wDvJpxMsCEu25bdfi87hnrktCXMKv67xhNJp2VDinzhGiH6o691uELMNjp7tS2tbxhxMZJGFdVMuKxVPj66gm3NFKWv",
            "xpub": "tpubDFswHqDuQ1P5TEjppsauEN1ymGQxEvpaHRiV5NoBco9ikaNjKm8QCyqay8Ag2q22N6fgXsRL4UCiUM83t5XfkrK22on49d5J3gz7v8wH4Lp"
        },
        {
            "address": "n3ngzXicoHPCXfjNQ77DciUTq4Ctm5wUwm",
            "index": 1,
            "path": "m/44'/1'/0'/0/1",
            "privkey": "cTnrHMJS2jiCmVhpQR5rcHRUEBhNtYWg5bs7untSGTd

In [9]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    """Convert the privkey string in a child key to an account object that bit or web3.py can use to transact"""
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key) 

In [10]:
eth_acc = priv_key_to_account(ETH,eth_PrivateKey)
btc_acc = priv_key_to_account(BTCTEST,btc_PrivateKey)

In [11]:
print(eth_acc)
print(btc_acc)

<PrivateKeyTestnet: mwaPpwmv4RHGKpHBdaJZf4RcTxEZBKDKqr>


In [12]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, recipient, amount):
    """create the raw, unsigned transaction that contains all metadata needed to transact"""
    global tx_data
    if coin ==ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        tx_data = {
            "to": recipient,
            "from": account.address,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address)
        }
        return tx_data

    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)]) 

In [13]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, recipient, amount):
    """call create_trx, sign the transaction, then send it to the designated network"""
    if coin == "eth": 
        tx_eth = create_tx(coin,account, recipient, amount)
        sign = account.signTransaction(trx_eth)
        result = w3.eth.sendRawTransaction(sign.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        tx_btctest= create_tx(coin,account,recipient,amount)
        sign_tx_btctest = account.sign_transaction(tx_btctest)
        from bit.network import NetworkAPI
        NetworkAPI.broadcast_tx_testnet(sign_tx_btctest)       
        return sign_tx_btctest
       

In [14]:
create_tx(BTCTEST,btc_acc,"mwaPpwmv4RHGKpHBdaJZf4RcTxEZBKDKqr", 0.00001)

'{"unspents":[{"amount":1000,"confirmations":7,"script":"76a914b02892efdb9fc0f37b4bab1efde62d47e137fef688ac","txid":"0bb1aca84c17dac09050389a8c5b242d3963a8589262db7b7b66ca56074bec2c","txindex":0,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295},{"amount":75948,"confirmations":7,"script":"76a914b02892efdb9fc0f37b4bab1efde62d47e137fef688ac","txid":"0bb1aca84c17dac09050389a8c5b242d3963a8589262db7b7b66ca56074bec2c","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["mwaPpwmv4RHGKpHBdaJZf4RcTxEZBKDKqr",1000],["mwaPpwmv4RHGKpHBdaJZf4RcTxEZBKDKqr",37800]]}'

In [15]:
send_tx(BTCTEST,btc_acc,'mwaPpwmv4RHGKpHBdaJZf4RcTxEZBKDKqr',0.00001)

'01000000022cec4b0756ca667b7bdb629258a863392d245b8c9a385090c0da174ca8acb10b010000006a4730440220317bc690f38df7038afe9c6818bef30e00523a4296e0d9f41289f6ceb73b35b0022038a65f1aa4ad459c7d12b0b4807555763e4ea5c1281b8e23c455032cf754c14d0121030efafe05f8ee8de6071c787cc1782e68fcfc0fdbdb9d3ce5fa7fa2d54607c0fcffffffff2cec4b0756ca667b7bdb629258a863392d245b8c9a385090c0da174ca8acb10b000000006a473044022035f2351e582c122ec9c9aa41ebad87d49679ca6e409dcdf2d6aa837f158fce2602204b3e57bce02b773db357d44f816c85cff6982cce00fcf04acc4c46c67c4f19c00121030efafe05f8ee8de6071c787cc1782e68fcfc0fdbdb9d3ce5fa7fa2d54607c0fcffffffff02e8030000000000001976a914b02892efdb9fc0f37b4bab1efde62d47e137fef688aca8930000000000001976a914b02892efdb9fc0f37b4bab1efde62d47e137fef688ac00000000'